In [1]:
import pandas as pd
import numpy as np

# Clean

In [2]:
df = pd.read_csv('recipe_data.csv')
df.head(1)

,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100164/chip-...,chip-truck-fries,4.4375,44 reviews,"['6 baking potatoes, cut into 1/4 inch slices'...",4.0,1 h,369 calories;,14.0,56.3,6.5,0,478.0


In [3]:
# check num of recipes
df.shape

(8331, 13)

In [7]:
# check nan
df.isnull().sum()

link                 0
title               16
ratings           1484
num_reviews       1486
ingredients         18
servings          1486
cook_time         1753
calorie           1512
fat_g             1512
carb_g            1512
protein_g         1512
cholesterol_mg    1512
sodium_mg         1513
dtype: int64

In [8]:
# check rows with no ratings
# df[df.ratings.isnull()]

In [9]:
# they also have other missing values so I will drop them
to_drop = df[df.ratings.isnull()].index.tolist()
df.drop(to_drop, inplace=True)

# check num of recipes
df.shape

(6847, 13)

In [11]:
# check rows with no nutrition values
# df[df.calorie.isnull()]

In [12]:
# I need nutrition values to optimize meal plans so I will drop them
to_drop = df[df.calorie.isnull()].index.tolist()
df.drop(to_drop, inplace=True)

# check num of recipes
df.shape

(6819, 13)

In [13]:
# check nan again
df.isnull().sum()

link                0
title               0
ratings             0
num_reviews         0
ingredients         1
servings            0
cook_time         267
calorie             0
fat_g               0
carb_g              0
protein_g           0
cholesterol_mg      0
sodium_mg           1
dtype: int64

In [15]:
# I need ingredients to group recipes so I will drop it
df[df.ingredients.isnull()]

,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
1544,https://www.allrecipes.com/recipe/188706/cabba...,cabbage-and-noodles,4.505465,293 reviews,NaN,4.0,45 m,447 calories;,19.1,53.1,17.5,90.0,616.0


In [16]:
df.drop(1544, inplace=True)

In [17]:
# check nan again
df.isnull().sum()

link                0
title               0
ratings             0
num_reviews         0
ingredients         0
servings            0
cook_time         267
calorie             0
fat_g               0
carb_g              0
protein_g           0
cholesterol_mg      0
sodium_mg           1
dtype: int64

In [18]:
# check data types
df.dtypes

link               object
title              object
ratings           float64
num_reviews        object
ingredients        object
servings          float64
cook_time          object
calorie            object
fat_g             float64
carb_g            float64
protein_g         float64
cholesterol_mg     object
sodium_mg          object
dtype: object

#### The data types of num_reviews, cook_time, calorie, cholesterol_mg and sodium_mg need to be changed to int or float. Then I will fill the missing values.

### Clean num_reviews

In [19]:
df.head(1)

,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100164/chip-...,chip-truck-fries,4.4375,44 reviews,"['6 baking potatoes, cut into 1/4 inch slices'...",4.0,1 h,369 calories;,14.0,56.3,6.5,0,478.0


In [20]:
# for num_reviews, remove " reviews"
df.num_reviews = df.num_reviews.apply(lambda x: x[:-8])

In [21]:
# some num_reviews contain "k", I will replace them with "000"
df_replace_k = df[df.num_reviews.str.contains('k')]
df_replace_k.num_reviews = df_replace_k.num_reviews.apply(lambda x: x[:-1] + '000')

# update rows in the original df
replace_idx = df_replace_k.index.tolist()
for idx in replace_idx: 
    df.loc[idx] = df_replace_k.loc[idx]
    
# change data type to int
df.num_reviews = df.num_reviews.astype('int')

# check results
print(df.num_reviews.dtypes)
df.loc[replace_idx[:5]]

/Users/fivecentsly/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


int64


,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
9,https://www.allrecipes.com/recipe/104850/black...,black-bean-and-corn-quesadillas,4.624080,1000,"['2 teaspoons olive oil', '3 tablespoons finel...",8.0,40 m,363 calories;,14.5,45.6,13.9,26,732.0
81,https://www.allrecipes.com/recipe/142027/sweet...,sweet-restaurant-slaw,4.764481,2000,"['1 (16 ounce) bag coleslaw mix', '2 tablespoo...",8.0,2 h 15 m,200 calories;,12.0,22.5,0.8,11,253
101,https://www.allrecipes.com/recipe/143069/super...,super-delicious-zuppa-toscana,4.821117,2000,"['1 pound bulk mild Italian sausage', '1 1/4 t...",6.0,1 h 25 m,554 calories;,32.6,45.8,19.8,99,2386
245,https://www.allrecipes.com/recipe/16715/vegeta...,vegetarian-chickpea-sandwich-filling,4.429831,1000,"['1 (19 ounce) can garbanzo beans, drained and...",3.0,20 m,259 calories;,5.8,43.5,9.3,2,576
1060,https://www.allrecipes.com/recipe/24833/black-...,black-bean-and-couscous-salad,4.679422,1000,"['1 cup uncooked couscous', '1 1/4 cups chicke...",8.0,35 m,253 calories;,5.8,41.1,10.3,0,415


### Clean cook_time

In [22]:
# for cook_time, subset the rows with no missing values
df_cooktime = df[df.cook_time.notnull()]

In [23]:
# define a function to convert cook_time to min
def convert_cooktime(x):
    cooktime = x.split()
    if 'h' in cooktime and 'm' in cooktime:
        x = int(cooktime[0]) * 60 + int(cooktime[2])
    elif 'h' in cooktime and 'm' not in cooktime:
        x = int(cooktime[0]) * 60
    else:
        x = int(cooktime[0])   
    return x

In [24]:
# apply function to cook_time
df_cooktime.cook_time = df_cooktime.cook_time.apply(convert_cooktime)

# update rows in the original df
replace_idx = df_cooktime.index.tolist()
for idx in replace_idx: 
    df.loc[idx] = df_cooktime.loc[idx]

# check results
print(df.cook_time.dtypes) # data type is object because there are missing values
df.loc[replace_idx[:5]]

object


,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100164/chip-...,chip-truck-fries,4.437500,44,"['6 baking potatoes, cut into 1/4 inch slices'...",4.0,60,369 calories;,14.0,56.3,6.5,0,478.0
1,https://www.allrecipes.com/recipe/100378/irish...,irish-bacon-and-cabbage-soup,4.536946,163,"['1/2 pound Irish bacon, diced', '2 large pota...",4.0,45,276 calories;,8.1,38.4,12.3,21,825.0
2,https://www.allrecipes.com/recipe/100402/quick...,quick-salmon-salad,4.491803,52,"['2 (6 ounce) cans pink salmon, drained', '1/2...",4.0,15,248 calories;,17.2,2.2,20.4,43,571.0
6,https://www.allrecipes.com/recipe/102242/bills...,bills-seasoned-ketchup,4.652174,22,"['1 (28 ounce) bottle ketchup', '2 teaspoons h...",45.0,10,19 calories;,0.1,4.7,0.4,0,201.0
7,https://www.allrecipes.com/recipe/103737/celer...,celery-salad,4.592233,83,"['3/4 cup sliced celery', '1/3 cup dried sweet...",2.0,40,150 calories;,3.0,26.5,4.1,< 1,304.0


In [25]:
# replace missing values of with 0 to indicate "unknown"
df.cook_time.fillna(0, inplace=True)
df.cook_time.dtypes

dtype('int64')

### Clean calorie

In [26]:
# for calorie, keep the number and change data type to float
df.calorie = df.calorie.apply(lambda x: float(x.split(' ')[0]))

# check results 
print(df.calorie.dtypes)
df.head(1)

float64


,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100164/chip-...,chip-truck-fries,4.4375,44,"['6 baking potatoes, cut into 1/4 inch slices'...",4.0,60,369.0,14.0,56.3,6.5,0,478.0


### Clean cholesterol_mg and sodium_mg

In [29]:
# some values of cholesterol_mg and sodium_mg are shown as "< 1", I will replace them with "0"
df.cholesterol_mg = df.cholesterol_mg.replace('< 1', 0)
df.sodium_mg = df.sodium_mg.replace('< 1', 0)

# change their data types to float
df.cholesterol_mg = df.cholesterol_mg.astype(float)
df.sodium_mg = df.sodium_mg.astype(float)

# check results 
print(df.cholesterol_mg.dtypes)
print(df.sodium_mg.dtypes)

float64
float64


In [30]:
# fill missing value with mean
df.sodium_mg = df.sodium_mg.fillna(np.round(df.sodium_mg.mean()))

In [31]:
df.sodium_mg.dtypes

dtype('float64')

### Clean ratings

In [32]:
# round ratings to two decimals
df.ratings = df.ratings.round(2)

# check results
df.head()

,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100164/chip-...,chip-truck-fries,4.44,44,"['6 baking potatoes, cut into 1/4 inch slices'...",4.0,60,369.0,14.0,56.3,6.5,0.0,478.0
1,https://www.allrecipes.com/recipe/100378/irish...,irish-bacon-and-cabbage-soup,4.54,163,"['1/2 pound Irish bacon, diced', '2 large pota...",4.0,45,276.0,8.1,38.4,12.3,21.0,825.0
2,https://www.allrecipes.com/recipe/100402/quick...,quick-salmon-salad,4.49,52,"['2 (6 ounce) cans pink salmon, drained', '1/2...",4.0,15,248.0,17.2,2.2,20.4,43.0,571.0
6,https://www.allrecipes.com/recipe/102242/bills...,bills-seasoned-ketchup,4.65,22,"['1 (28 ounce) bottle ketchup', '2 teaspoons h...",45.0,10,19.0,0.1,4.7,0.4,0.0,201.0
7,https://www.allrecipes.com/recipe/103737/celer...,celery-salad,4.59,83,"['3/4 cup sliced celery', '1/3 cup dried sweet...",2.0,40,150.0,3.0,26.5,4.1,0.0,304.0


### Clean title

In [33]:
# change the format of title
df.title = df.title.apply(lambda x: x.replace('-', ' ').title())

# check results
df.head()

,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100164/chip-...,Chip Truck Fries,4.44,44,"['6 baking potatoes, cut into 1/4 inch slices'...",4.0,60,369.0,14.0,56.3,6.5,0.0,478.0
1,https://www.allrecipes.com/recipe/100378/irish...,Irish Bacon And Cabbage Soup,4.54,163,"['1/2 pound Irish bacon, diced', '2 large pota...",4.0,45,276.0,8.1,38.4,12.3,21.0,825.0
2,https://www.allrecipes.com/recipe/100402/quick...,Quick Salmon Salad,4.49,52,"['2 (6 ounce) cans pink salmon, drained', '1/2...",4.0,15,248.0,17.2,2.2,20.4,43.0,571.0
6,https://www.allrecipes.com/recipe/102242/bills...,Bills Seasoned Ketchup,4.65,22,"['1 (28 ounce) bottle ketchup', '2 teaspoons h...",45.0,10,19.0,0.1,4.7,0.4,0.0,201.0
7,https://www.allrecipes.com/recipe/103737/celer...,Celery Salad,4.59,83,"['3/4 cup sliced celery', '1/3 cup dried sweet...",2.0,40,150.0,3.0,26.5,4.1,0.0,304.0


In [48]:
# check duplicates
df.link.duplicated().sum()

995

In [49]:
# drop duplicated links
df.drop_duplicates('link', inplace=True)

In [50]:
df.link.duplicated().sum()

0

In [54]:
df[df.title.duplicated()]

,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
2148,https://www.allrecipes.com/recipe/14090/mexica...,Mexican Rice,4.26,185,"['3 tablespoons vegetable oil', '2/3 cup diced...",8.0,50,199.0,5.6,33.8,3.4,0.0,809.0
2932,https://www.allrecipes.com/recipe/25145/mexica...,Mexican Casserole,4.40,721,"['2 tablespoons vegetable oil', '3/4 pound cub...",5.0,30,384.0,16.6,34.0,26.8,59.0,1286.0
3112,https://www.allrecipes.com/recipe/35304/pico-d...,Pico De Gallo,4.40,352,"['1 medium tomato, diced', '1 onion, finely ch...",4.0,50,21.0,0.1,4.7,0.8,0.0,76.0


In [55]:
df[df.title == 'Mexican Rice']

,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
1451,https://www.allrecipes.com/recipe/22884/mexica...,Mexican Rice,4.56,109,"['1 pound lean ground beef', '1 onion, diced',...",6.0,55,510.0,18.3,59.1,28.3,74.0,1294.0
2148,https://www.allrecipes.com/recipe/14090/mexica...,Mexican Rice,4.26,185,"['3 tablespoons vegetable oil', '2/3 cup diced...",8.0,50,199.0,5.6,33.8,3.4,0.0,809.0


In [58]:
df.shape

(5823, 13)

In [57]:
# save df
df.to_csv('recipe_clean.csv', index=False)

#### Considered duplidated titles, I will not use title as an ID in the following analysis.